In [ ]:
!nvidia-smi

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split



In [ ]:
!pip install wget
import wget
wget.download('https://storage.googleapis.com/wandb_datasets/nature_12K.zip')
!unzip /content/nature_12K.zip

#**Dataset creation** 
Created dataloaders with batch size=32(I dont vary batch size as it was creating out of memory issue with higher values)
Outputs show the size of one batch(used for debugging)

In [ ]:
def dataset_creation(data_augmentation=True):
  if data_augmentation:
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
        transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #These values are standard values used in imagenet competition
    ])
  else:
        train_transforms = transforms.Compose([
        transforms.RandomResizedCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #These values are standard values used in imagenet competition
    ])
  
  val_transforms = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      #These values are standard values used in imagenet competition
      #target_transform=None for creating a non-augmented dataset
  ])

  # Create training and validation datasets
  train_dataset = ImageFolder('/content/inaturalist_12K/train', transform=train_transforms)
  train_size = int(0.8 * len(train_dataset))
  val_size = len(train_dataset) - train_size
  train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
  test_dataset = ImageFolder('/content/inaturalist_12K/val', transform=val_transforms)
   
  # Create training and validation data loaders
  #Need to add this in the main function and make batch_size hyperparameter from 32 to 64
  BATCH_SIZE=32
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
  val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
  return train_loader,val_loader

train_loader,val_loader=dataset_creation()

print(f"Data loader: {train_loader,val_loader}")
print(f"Length of train dataset loader:{len(train_loader)}")

print(f"Data loader: {train_loader,val_loader}")
print(f"Length of val dataset loader:{len(val_loader)}")
train_features_batch,train_labels_batch=next(iter(train_loader))
train_features_batch.shape,train_labels_batch.shape

Data loader: (<torch.utils.data.dataloader.DataLoader object at 0x7fdbbb0746d0>, <torch.utils.data.dataloader.DataLoader object at 0x7fdbbb074550>)
Length of train dataset loader:250
Data loader: (<torch.utils.data.dataloader.DataLoader object at 0x7fdbbb0746d0>, <torch.utils.data.dataloader.DataLoader object at 0x7fdbbb074550>)
Length of val dataset loader:63


(torch.Size([32, 3, 224, 224]), torch.Size([32]))

# **Visualizing dataset**
This code block is also for debugging the transforms

In [ ]:
'''import warnings
warnings.filterwarnings("ignore")
image,label=[],[]
for i in range(2):
 a,b=train_dataset[3000+20*i]
 image.append(a)
 label.append(b)

class_names=train_dataset.classes
print(f"class_names are: {class_names}")
class_to_idx=train_dataset.class_to_idx
print(f"class_to_idx: {class_to_idx}")
for i in range(len(image)):
 print(f"image shape is:{image[i].shape} and class name:{class_names[label[i]]}")


#We get cropped and trasformed images!!

for i in range(len(image)):
 plt.imshow(image[i].permute(1, 2, 0))
 plt.title(f"Class: {class_names[label[i]]}")
 plt.axis(False)
 plt.show()'''

In [ ]:
!pip install --upgrade wandb
!wandb login aa5afea12b4fda1e7f8310b597eb17c73d1176d2 #my API key for wandb login 
import wandb

# **Model**
Created MyModel class which is my CNN architecture
(I have repeated the same class, again for )

In [ ]:
class MyModel(nn.Module):
    def __init__(self, num_filter=[16,16,16,16,16], kernel_size=[5, 5, 5, 5, 5], stride=[1, 1, 1, 1, 1], padding=[1, 1, 1, 1, 1], activation='relu',batch_normalization='True',dropout=0.2):
        super(MyModel, self).__init__()
        self.num_filter = num_filter
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.activation = activation
        self.batch_normalization=batch_normalization

        self.conv1 = nn.Conv2d(3, num_filter[0], kernel_size=kernel_size[0], stride=stride[0], padding=padding[0])
        self.bn1 = nn.BatchNorm2d(num_filter[0])
        self.dropout1 = nn.Dropout(dropout)
        self.conv2 = nn.Conv2d(num_filter[0], num_filter[1], kernel_size=kernel_size[1], stride=stride[1], padding=padding[1])
        self.bn2 = nn.BatchNorm2d(num_filter[1])
        self.dropout2 = nn.Dropout(dropout)
        self.conv3 = nn.Conv2d(num_filter[1], num_filter[2], kernel_size=kernel_size[2], stride=stride[2], padding=padding[2])
        self.bn3 = nn.BatchNorm2d(num_filter[2])
        self.dropout3 = nn.Dropout(dropout)
        self.conv4 = nn.Conv2d(num_filter[2], num_filter[3], kernel_size=kernel_size[3], stride=stride[3], padding=padding[3])
        self.bn4 = nn.BatchNorm2d(num_filter[3])
        self.dropout4 = nn.Dropout(dropout)
        self.conv5 = nn.Conv2d(num_filter[3], num_filter[4], kernel_size=kernel_size[4], stride=stride[4], padding=padding[4])
        self.bn5 = nn.BatchNorm2d(num_filter[4])
        self.dropout5 = nn.Dropout(dropout)

        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'gelu':
            self.activation = nn.GELU()
        elif activation == 'elu':
            self.activation = nn.ELU()
        elif activation == 'silu':
            self.activation = nn.SiLU()
        
        self.pooling = nn.MaxPool2d(kernel_size=2)
        self.avgpool=nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.dense=nn.Linear(num_filter[4],256)
        self.fc = nn.Linear(256, 10)


    def forward(self, x):
        x = self.conv1(x)
        if self.batch_normalization: x = self.bn1(x)         
        x = self.activation(x)
        x=self.dropout1(x)
        x = self.pooling(x)
      
        x = self.conv2(x)
        if self.batch_normalization: x = self.bn2(x)
        x = self.activation(x)
        x=self.dropout2(x)
        x = self.pooling(x)

        x = self.conv3(x)
        if self.batch_normalization: x = self.bn3(x)
        x = self.activation(x)
        x=self.dropout3(x)
        x = self.pooling(x)

        x = self.conv4(x)
        if self.batch_normalization: x = self.bn4(x)
        x = self.activation(x)
        x=self.dropout4(x)
        x = self.pooling(x)

        x = self.conv5(x)
        if self.batch_normalization: x = self.bn5(x)
        x = self.activation(x)
        x=self.dropout5(x)
        x = self.pooling(x)

        x = self.avgpool(x)#read why cant we use maxpooling here
        x = torch.flatten(x, 1)
        x = self.dense(x)
        x = self.fc(x)

        return x
    


In [ ]:
train_loader,val_loader=dataset_creation()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Script used for debugging
'''
import torch
torch.cuda.empty_cache()
if torch.cuda.is_available():
  print('Using GPU...!')
else:
  print('Using CPU...!(terminate the runtime and restart using GPU)')

# Initialize model, loss function, and optimizer

model = MyModel( num_filter=[32,64,128,256,512],kernel_size=[3,3,5,5,7] ,activation='elu').to(device)
model.fit_train()
summary(model,(3,224,224))'''

In [ ]:
!pip install --upgrade wandb
!wandb login aa5afea12b4fda1e7f8310b597eb17c73d1176d2 #my API key for wandb login 
import wandb

# **Sweeping over the Hyperparameters**

In [ ]:
from torchsummary import summary
sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'activation': {
            'values': ['relu','gelu','elu','silu']
        },
        'num_filter': {
            'values': [[256,256,256,256,256],[128,128,128,128,128],
                       [64,128,256,512,1024],[64,64, 64,64,64]]
        },        
        'kernel_size':{
            'values':[[3,3,3,5,5],[5,5,5,5,5],[3,3,3,3,3]]
        },
        'dropout':{
            'values':[0.2,0.4]
        },
        'batch_norm':{
            'values':[True,False]
        },
        'data_augmentation':{
            'values':[True,False]
        },
        
    }
}

sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS6910-Assignment2")

def sweep_train():
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'activation':'relu',
      'num_filter':[256, 256, 256, 256, 256],
      'kernel_size':[3,3,3,3,3],
      'dropout':0.2,
      'batch_norm':True,
      'data_augmentation':True,
  }

  # Initialize a new wandb run
  wandb.init(project='CS6910-Assignment2', entity='shreyashgadgil007',config=config_defaults)
  wandb.run.name = 'act:'+ str(wandb.config.activation)+' ;filter:'+str(wandb.config.num_filter)+ ' ;ker:'+str(wandb.config.kernel_size)+ ' ;drop:'+str(wandb.config.dropout)+' ;b_n:'+str(wandb.config.batch_norm)+' ;d_a:'+str(wandb.config.data_augmentation)

  
  config = wandb.config
  activation = config.activation
  num_filter = config.num_filter
  kernel_size = config.kernel_size
  dropout = config.dropout
  batch_norm = config.batch_norm
  data_augmentation = config.data_augmentation
  # Model training here
  #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = MyModel( num_filter=num_filter,kernel_size=kernel_size, activation=activation,batch_normalization=batch_norm).to(device)
  summary(model,(3,224,224))
  train_loader,val_loader=dataset_creation(data_augmentation)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(),lr=0.0003, betas=(0.9,0.9999))
   # Train the model
  num_epochs = 15
  for epoch in range(num_epochs):
          # Set to training mode
          model.train()

          running_loss = 0.0
          running_corrects = 0
          for inputs, labels in train_loader:
              inputs, labels = inputs.to(device), labels.to(device)

              optimizer.zero_grad()

              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)

              loss.backward()
              optimizer.step()

              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

          train_epoch_loss = running_loss / len(train_loader.dataset)
          train_epoch_acc = (running_corrects.double() / len(train_loader.dataset))*100

          # Set to evaluation mode
          model.eval()

          running_loss = 0.0
          running_corrects = 0
          with torch.no_grad():
              for inputs, labels in val_loader:
                  inputs, labels = inputs.to(device), labels.to(device)

                  # Forward pass
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)

                  # Statistics
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

          val_epoch_loss = running_loss / len(val_loader.dataset)
          val_epoch_acc = (running_corrects.double() / len(val_loader.dataset))*100
          
          print(f"Epoch {epoch+1}/{num_epochs}--> Training_Loss:{train_epoch_loss:.2f}; Train_Accuracy:{train_epoch_acc:.2f}; Validation_Loss:{val_epoch_loss:.2f}; Val_Accuracy:{val_epoch_acc:.2f}")
          wandb.log({"train_loss":train_epoch_loss,"train_accuracy": train_epoch_acc,"val_loss":val_epoch_loss,"val_accuracy":val_epoch_acc},)
          #emptying the cache after one complete run
          if epoch==num_epochs-1:
            torch.cuda.empty_cache()


#RUNNING THE SWEEP
wandb.agent(sweep_id, function=sweep_train, count=120)


Create sweep with ID: ifwvwpk1
Sweep URL: https://wandb.ai/shreyashgadgil007/CS6910-Assignment2/sweeps/ifwvwpk1


wandb: Agent Starting Run: sti2tdqc with config:
wandb: 	activation: elu
wandb: 	batch_norm: True
wandb: 	data_augmentation: True
wandb: 	dropout: 0.4
wandb: 	kernel_size: [3, 3, 3, 3, 3]
wandb: 	num_filter: [256, 256, 256, 256, 256]
wandb: Currently logged in as: shreyashgadgil007. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 256, 224, 224]           7,168
       BatchNorm2d-2        [-1, 256, 224, 224]             512
               ELU-3        [-1, 256, 224, 224]               0
           Dropout-4        [-1, 256, 224, 224]               0
         MaxPool2d-5        [-1, 256, 112, 112]               0
            Conv2d-6        [-1, 256, 112, 112]         590,080
       BatchNorm2d-7        [-1, 256, 112, 112]             512
               ELU-8        [-1, 256, 112, 112]               0
           Dropout-9        [-1, 256, 112, 112]               0
        MaxPool2d-10          [-1, 256, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         590,080
      BatchNorm2d-12          [-1, 256, 56, 56]             512
              ELU-13          [-1, 256, 56, 56]               0
          Dropout-14          [-1, 256,

train_accuracy,▁▂▃▃▄▄▄▅▅▅▇▇▇██
train_loss,█▇▅▅▅▅▄▄▃▃▂▂▂▁▁
val_accuracy,▃▃▄▅▄▄▁▂▄▂█▄▆▇▅
val_loss,▅▆▃▄▄▄█▅▃█▁▄▃▃▆
train_accuracy,26.44081
train_loss,2.06435
val_accuracy,22.05
val_loss,2.22723


wandb: Agent Starting Run: dhrm422q with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	data_augmentation: False
wandb: 	dropout: 0.4
wandb: 	kernel_size: [5, 5, 5, 5, 5]
wandb: 	num_filter: [64, 128, 256, 512, 1024]
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 222, 222]           4,864
       BatchNorm2d-2         [-1, 64, 222, 222]             128
              ReLU-3         [-1, 64, 222, 222]               0
           Dropout-4         [-1, 64, 222, 222]               0
         MaxPool2d-5         [-1, 64, 111, 111]               0
            Conv2d-6        [-1, 128, 109, 109]         204,928
       BatchNorm2d-7        [-1, 128, 109, 109]             256
              ReLU-8        [-1, 128, 109, 109]               0
           Dropout-9        [-1, 128, 109, 109]               0
        MaxPool2d-10          [-1, 128, 54, 54]               0
           Conv2d-11          [-1, 256, 52, 52]         819,456
      BatchNorm2d-12          [-1, 256, 52, 52]             512
             ReLU-13          [-1, 256, 52, 52]               0
          Dropout-14          [-1, 256,

train_accuracy,▁▃▃▄▄▅▅▆▆▆▇████
train_loss,█▆▆▅▅▄▄▃▃▃▂▂▂▁▁
val_accuracy,▂▄▄▁▄▅▄▆▆▆▄█▄▄█
val_loss,▇▆▆█▅▅▃▃▂▂█▁▂▆▆
train_accuracy,35.36692
train_loss,1.82519
val_accuracy,26.05
val_loss,2.15366


wandb: Agent Starting Run: jnnx7xwz with config:
wandb: 	activation: relu
wandb: 	batch_norm: False
wandb: 	data_augmentation: True
wandb: 	dropout: 0.2
wandb: 	kernel_size: [5, 5, 5, 5, 5]
wandb: 	num_filter: [64, 128, 256, 512, 1024]
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 222, 222]           4,864
              ReLU-2         [-1, 64, 222, 222]               0
           Dropout-3         [-1, 64, 222, 222]               0
         MaxPool2d-4         [-1, 64, 111, 111]               0
            Conv2d-5        [-1, 128, 109, 109]         204,928
              ReLU-6        [-1, 128, 109, 109]               0
           Dropout-7        [-1, 128, 109, 109]               0
         MaxPool2d-8          [-1, 128, 54, 54]               0
            Conv2d-9          [-1, 256, 52, 52]         819,456
             ReLU-10          [-1, 256, 52, 52]               0
          Dropout-11          [-1, 256, 52, 52]               0
        MaxPool2d-12          [-1, 256, 26, 26]               0
           Conv2d-13          [-1, 512, 24, 24]       3,277,312
             ReLU-14          [-1, 512,

# **Implementating best model for the test dataset and logging it to wandb**

In [ ]:
def dataset_creation(data_augmentation=True):
  if data_augmentation:
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
        transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
  else:
        train_transforms = transforms.Compose([
        transforms.RandomResizedCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
  
  val_transforms = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  # Create training and test datasets
  train_dataset = ImageFolder('/content/inaturalist_12K/train', transform=train_transforms)
  train_size = int(0.8 * len(train_dataset))
  val_size = len(train_dataset) - train_size
  train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
  test_dataset = ImageFolder('/content/inaturalist_12K/val', transform=val_transforms)
   
  # Create training and test data loaders
  BATCH_SIZE=32
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
  return train_loader,test_loader

train_loader,test_loader=dataset_creation()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torchsummary import summary
sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'activation': {
            'values': ['silu']
        },
        'num_filter': {
            'values': [[64,128,256,512,1024]]
        },        
        'kernel_size':{
            'values':[[5,5,5,5,5]]
        },
        'dropout':{
            'values':[0.4]
        },
        'batch_norm':{
            'values':[False]
        },
        'data_augmentation':{
            'values':[False]
        },
        
    }
}

sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS6910-Assignment2")

def sweep_train():
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'activation':'relu',
      'num_filter':[256, 256, 256, 256, 256],
      'kernel_size':[3,3,3,3,3],
      'dropout':0.2,
      'batch_norm':True,
      'data_augmentation':True,
  }

  # Initialize a new wandb run
  wandb.init(project='CS6910-Assignment2', entity='shreyashgadgil007',config=config_defaults)
  wandb.run.name = 'Testing_on_the_best_combination'
  
  config = wandb.config
  activation = config.activation
  num_filter = config.num_filter
  kernel_size = config.kernel_size
  dropout = config.dropout
  batch_norm = config.batch_norm
  data_augmentation = config.data_augmentation
  # Model training here
  #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = MyModel( num_filter=num_filter,kernel_size=kernel_size, activation=activation,batch_normalization=batch_norm).to(device)
  summary(model,(3,224,224))
  train_loader,val_loader=dataset_creation(data_augmentation)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(),lr=0.0003, betas=(0.9,0.9999))
   # Train the model
  num_epochs = 15
  for epoch in range(num_epochs):
          # Set to training mode
          model.train()

          running_loss = 0.0
          running_corrects = 0
          for inputs, labels in train_loader:
              inputs, labels = inputs.to(device), labels.to(device)

              optimizer.zero_grad()

              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)

              loss.backward()
              optimizer.step()

              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

          train_epoch_loss = running_loss / len(train_loader.dataset)
          train_epoch_acc = (running_corrects.double() / len(train_loader.dataset))*100

          # Set to evaluation mode
          model.eval()

          running_loss = 0.0
          running_corrects = 0
          with torch.no_grad():
              for inputs, labels in val_loader:
                  inputs, labels = inputs.to(device), labels.to(device)

                  # Forward pass
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)

                  # Statistics
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

          val_epoch_loss = running_loss / len(val_loader.dataset)
          val_epoch_acc = (running_corrects.double() / len(val_loader.dataset))*100
          
          print(f"Epoch {epoch+1}/{num_epochs}--> Training_Loss:{train_epoch_loss:.2f}; Train_Accuracy:{train_epoch_acc:.2f}; Validation_Loss:{val_epoch_loss:.2f}; Val_Accuracy:{val_epoch_acc:.2f}")
          wandb.log({"train_loss":train_epoch_loss,"train_accuracy": train_epoch_acc,"val_loss":val_epoch_loss,"val_accuracy":val_epoch_acc},)
          #emptying the cache after one complete run
          if epoch==num_epochs-1:
            torch.cuda.empty_cache()


#RUNNING THE SWEEP
wandb.agent(sweep_id, function=sweep_train, count=1)


# **Plotting grid for test data and predictions**
Made changes in fit_train() to get return values as output images and correct labels, predicted labels of test data for last epoch.

In [ ]:
def fit_train():
      criterion = nn.CrossEntropyLoss()
      optimizer = optim.Adam(model.parameters(), lr=0.0003, betas=(0.9,0.9999))

      # Train the model
      num_epochs = 15
      out_img=[]
      pred_label_o=[]
      actual_output=[]
      for epoch in range(num_epochs):
          # Set to training mode
          model.train()

          running_loss = 0.0
          running_corrects = 0
          for inputs, labels in train_loader:
              inputs, labels = inputs.to(device), labels.to(device)

              optimizer.zero_grad()

              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)
              
              loss.backward()
              optimizer.step()

              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

          train_epoch_loss = running_loss / len(train_loader.dataset)
          train_epoch_acc = (running_corrects.double() / len(train_loader.dataset))*100

          # Evaluate on test set
          model.eval()

          running_loss = 0.0
          running_corrects = 0
          with torch.no_grad():
              for inputs, labels in test_loader:
                  inputs, labels = inputs.to(device), labels.to(device)

                  # Forward pass
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)
                  
                  #DATA LOGGING FOR VISUALIZATION
                  if epoch == num_epochs-1:
                     out_img.append(inputs)
                     pred_label_o.append(int(preds[0]))
                     actual_output.append(int(labels[0]))
                     print(out_img)
                     print(pred_label_o)
                     print(actual_output)
                  # Statistics
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

          val_epoch_loss = running_loss / len(test_loader.dataset)
          val_epoch_acc = (running_corrects.double() / len(test_loader.dataset))*100
          
          print(f"Epoch {epoch+1}/{num_epochs}--> Training_Loss:{train_epoch_loss:.2f}; Train_Accuracy:{train_epoch_acc:.2f}; Validation_Loss:{val_epoch_loss:.2f}; Val_Accuracy:{val_epoch_acc:.2f}")
          
          torch.cuda.empty_cache()
      return out_img,pred_label_o,actual_output


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  print('Using GPU...!')
else:
  print('Using CPU...!(terminate the runtime and restart using GPU)')

# Initialize model, loss function, and optimizer

model = MyModel( num_filter=[64,128,256,512,1024],kernel_size=[5,5,5,5,5] ,activation='silu').to(device)
out_img,pred_label_o,actual_output=fit_train()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
image,label=[],[]
f_image = out_img[0]
f_image_copy = f_image.clone()
np_image = f_image_copy.cpu().numpy()

for i in range(30):
 a=np_image[i]
 image.append(a)
labels={0:'Amphibia',1:'Animalia',2:'Ararchnida',3:'Aves',4:'Fungi',5:'Insecta',6:'Mammalia',7:'Mollusca',8:'Plantae',9:'Reptilia'}

xyz = plt.figure(figsize=(20,20))
for i in range(len(image)):
 xyz.add_subplot(10, 3, i+1)
 image[i] = image[i].transpose(1, 2, 0)
 plt.imshow(image[i])
 plt.xlabel('correct : '+ labels[actual_output[i]])
 plt.ylabel('predicted : '+labels[pred_labels_o[i]])
 plt.xticks([])
 plt.yticks([])
 plt.axis(False)
 plt.grid(0)
xyz.tight_layout()
plt.show()